In [1]:
import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
from cupyx.profiler import benchmark
from matplotlib import pyplot as plt

from qwak_cupy.QWAK import QWAK as CQWAK
from qwak.QWAK import QWAK as QWAK

# Simple Benchmark

In [ ]:
t = 0.5
n = 1000
graph = nx.cycle_graph(n)
initNodes = [n//2]

qw = CQWAK(graph)
qw.runWalk(t, initNodes)
print(qw.getProbVec())
print(type(qw.getProbVec()))